In [ ]:
import os
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Coron3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from glob import glob
import astropy.io.fits as fits
import numpy as np
import time
import pandas as pd
import json
import pickle
from json import JSONEncoder
from collections import defaultdict
from CoronPipeline import MyCoron3Pipeline

In [ ]:
os.environ["CRDS_PATH"] = '/home/sarperyn/crds_cache/jwst_ops'
os.environ["CRDS_SERVER_URL"] = 'https://jwst-crds.stsci.edu'

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [20,15]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

In [ ]:
import jwst
print(jwst.__version__)

In [ ]:
PROPOSAL_ID = '1536'
INSTRUME = 'NIRCAM'
#DETECTOR = 'nrca2'

# Pipeline things



<div class="alert alert-block alert-info">
Now we have the fits files. We will create our coronagrahic images using the JWST pipeline. Our data has the suffix <b>.rateints</b> just like we needed. 
While processing coronagraphy data, the common procedure is using the following pipeline:
    <ul> 
        <li> calwebb_detector1 $\rightarrow$ <strong>from jwst.pipeline import Detector1Pipeline</strong>
        <li> calwebb_image2  $\rightarrow$ <strong>from jwst.pipeline import Image2Pipeline</strong>
        <li> calwebb_coron3  $\rightarrow$ <strong>from jwst.pipeline import Coron3Pipeline</strong>
    </ul>
</div>

We have the outputs of the first stage. They all have the `rateints.fits` suffix. By passing them through the second pipeline we are going to end up with `calints.fits` files which is a last step before getting the coron products.

In [ ]:
# Path containing the rateints data
directory = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/{INSTRUME}/{PROPOSAL_ID}/mastDownload/JWST/'

In [ ]:
directory

### Stage2

In [ ]:
def get_exptypes(fits_):
    
    for file in range(len(fits_)):
        
        f = fits.open(fits_[file])
        exp_type = f[0].header['EXP_TYPE']
        print(exp_type)

In [ ]:
def runimg2(filename, output_dir):
    
    img2 = Image2Pipeline()
    img2.output_dir = output_dir
    img2.save_results = True
    img2(filename)

In [ ]:
def delete_files(files):
    for file in files:

        if os.path.isfile(file):
            os.remove(file)
            print("File has been deleted")
        else:
            print("File does not exist")

In [ ]:
rateints_files = glob(os.path.join(directory, '**/*nrca2_rateints.fits'))

In [ ]:
batch_size = 4

In [ ]:
for i in range(0,len(rateints_files),batch_size):
    
    for f in rateints_files[i:i+batch_size]:
        
        
        output_dir = '/'.join(f.split('/')[:-1]) + '/' 
        runimg2(f,output_dir)
    time.sleep(1)

In [ ]:
calints_data = glob(os.path.join(directory, '**/*nrcalong_calints.fits'))

In [ ]:
calints1 = fits.open(calints_data[0])
calints1.info()

### Stage 3

The stage 3 is kinda different than the other steps. To run this step we need an association (ASN) file. Thus, we need some functions to create ASN files.

In [ ]:
len(calints_data)

In [ ]:
from jwst.associations.mkpool import mkpool
from jwst.associations.lib.rules_level3 import Asn_Lv3Coron
from jwst.associations import AssociationPool, AssociationRegistry
from jwst.associations.mkpool import from_cmdline, mkpool
from jwst.associations import generate
from astropy.table import Table

In [ ]:
pool = mkpool(calints_data)

In [ ]:
pool_df = pool.to_pandas()
pool_df.to_csv(f'calints_{INSTRUME}_{PROPOSAL_ID}_pool.csv',index=False)

In [ ]:
def t_path(partial_path):
    __file__ = '/home/sarperyn/sarperyurtseven/ProjectFiles/notebooks/asn_coron_rule.py'
    test_dir = os.path.dirname(__file__)
    return os.path.join(test_dir, partial_path)

In [ ]:
registry = AssociationRegistry([t_path('asn_coron_rule.py')],
                              include_default=False)

In [ ]:
asns = generate(pool,registry)

In [ ]:
len(asns)

In [ ]:
asns

In [ ]:
class FileEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

In [ ]:
def get_asns(asns):
    
    asn_list = []
    for idx,asn in enumerate(asns):
        asn_dict = {}
        
        for i,j in zip(asn.keys(),asn.values()):
            asn_dict[i] = j
            
        asn_list.append(asn_dict)
        
        
        # I couldn't figure out how to serialize the abc.AsnCoron_Lv3 object and save it as a json file
        # For a temporary solution we will have a list that contains asn files as dictionaries
        # In this way, we can easily run the pipeline
        '''
        FileEncoder().encode(asn_dict)
        with open(f'asnfile_{idx}.json','w') as output:
            encodedJSON = json.dump(asn_dict,output,indent=4,cls=FileEncoder)
        '''
        
    return asn_list
            

In [ ]:
def runcoron(filename, output_dir):

    coron = MyCoron3Pipeline()
    coron.output_dir = output_dir
    coron.save_results = True
    coron.process(filename)

In [ ]:
def get_stage3_products(asns,directory):
    
    for t in range(len(asns)):

        asn_dict = {}
        for i,j in zip(asns[t].keys(),asns[t].values()):
            asn_dict[i] = j
            
        runcoron(asn_dict,directory)
        
        

In [ ]:
print(len(asns[0]['products'][0]['members']))
print(len(asns[1]['products'][0]['members']))
#print(len(asns[2]['products'][0]['members']))

In [ ]:
get_stage3_products(asns,directory)